#  Import and def functions

In [1]:
import pandas as pd
from python_calamine import CalamineWorkbook
import numbers
import os
import glob

In [2]:
def extract_year(filename):
    for part in filename.split('/'):
        if part.isdigit() and len(part) == 4:
            return part
    return ''.join(filter(str.isdigit, filename))[-4:]

In [3]:
estados = [
   'Rondônia', 'Acre', 'Amazonas', 'Roraima', 'Pará', 'Amapá', 'Tocantins', 
   'Maranhão', 'Piauí', 'Ceará', 'Rio Grande do Norte', 'Paraíba', 'Pernambuco', 
   'Alagoas', 'Sergipe', 'Bahia', 'Minas Gerais', 'Espírito Santo', 'Rio de Janeiro', 
   'São Paulo', 'Paraná', 'Santa Catarina', 'Rio Grande do Sul', 'Mato Grosso do Sul', 
   'Mato Grosso', 'Goiás', 'Distrito Federal'
]

# TEA estados

In [8]:
try:
    df_existing = pd.read_csv("data-out/data-TEA-estados.csv", index_col='Ano')
except FileNotFoundError:
    df_existing = pd.DataFrame()

# Process all Excel files in data directory
for file in sorted(glob.glob("data-src/*.xlsx")):
    year = extract_year(file)
    
    if year in df_existing.columns:
        print(f"Data for year {year} already exists in the dataset.")
        continue

    print(f"Processing file: {file} for year {year}")
    wb = CalamineWorkbook.from_path(file)
    
    # Define sheet names and columns once
    aba_comum = "1.44" if year == "2023" else "1.43"
    aba_exclusiva = "1.50" if year == "2023" else "1.49"
    columns_to_keep = ["Regiao", "Estado", "Municipio", "cod_Municipio", "TEA"]

    def process_df(data):
        df = pd.DataFrame(data[9:], columns=data[7])
        df.columns.values[0:4] = ["Regiao", "Estado", "Municipio", "cod_Municipio"]
        df = df.rename(columns={df.columns[13]: "TEA"})
        return df[columns_to_keep]

    # Process both sheets
    df_comum = process_df(wb.get_sheet_by_name(aba_comum).to_python())
    df_exclusiva = process_df(wb.get_sheet_by_name(aba_exclusiva).to_python())
    
    # Convert TEA to numeric, replacing non-numeric values with 0
    df_comum['TEA'] = pd.to_numeric(df_comum['TEA'], errors='coerce').fillna(0)
    df_exclusiva['TEA'] = pd.to_numeric(df_exclusiva['TEA'], errors='coerce').fillna(0)
    
    # Group by all columns except TEA and sum the TEA values
    group_columns = ['Regiao', 'Estado', 'Municipio', 'cod_Municipio']
    df = pd.concat([df_comum, df_exclusiva]).groupby(group_columns, as_index=False)['TEA'].sum()

    # Filter states according to the predefined list and remove empty state entries
    df_estados = df[df['Estado'].str.strip().isin(estados)]
    df_estados = df_estados[df_estados['Estado'].str.strip() != '']
    df_estados = df_estados.loc[:, ["Estado", "TEA"]]
    
    # Create pivot table with states as index
    df_pivot = df_estados.pivot_table(index='Estado', values='TEA', aggfunc='first')
    
    if df_existing.empty:
        df_existing = df_pivot
        df_existing.columns = [year]
    else:
        df_existing[year] = df_pivot['TEA']
    
    # Export to CSV
    df_existing.to_csv("data-out/data-TEA-estados.csv")

print(file)

Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2014.xlsx for year 2014
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2015.xlsx for year 2015
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2016.xlsx for year 2016
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2017.xlsx for year 2017
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2018.xlsx for year 2018
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2019.xlsx for year 2019
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2020.xlsx for year 2020
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2021.xlsx for year 2021
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2022.xlsx for year 2022
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx for year 2023
data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx


# TEA municipios

In [9]:
try:
    df_existing = pd.read_csv("data-out/data-TEA-matriculas.csv", index_col='Ano')
except FileNotFoundError:
    df_existing = pd.DataFrame()

for file in sorted(glob.glob("data-src/*.xlsx")):
    year = extract_year(file)
    
    if year in df_existing.columns:
        print(f"Data for year {year} already exists in the dataset.")
        continue

    print(f"Processing file: {file} for year {year}")
    wb = CalamineWorkbook.from_path(file)
    
    # Define sheet names and columns once
    aba_comum = "1.44" if year == "2023" else "1.43"
    aba_exclusiva = "1.50" if year == "2023" else "1.49"
    columns_to_keep = ["Regiao", "Estado", "Municipio", "cod_Municipio", "TEA"]

    def process_df(data):
        df = pd.DataFrame(data[9:], columns=data[7])
        df.columns.values[0:4] = ["Regiao", "Estado", "Municipio", "cod_Municipio"]
        df = df.rename(columns={df.columns[13]: "TEA"})
        return df[columns_to_keep]

    # Process both sheets
    df_comum = process_df(wb.get_sheet_by_name(aba_comum).to_python())
    df_exclusiva = process_df(wb.get_sheet_by_name(aba_exclusiva).to_python())
    
    # Convert TEA to numeric, replacing non-numeric values with 0
    df_comum['TEA'] = pd.to_numeric(df_comum['TEA'], errors='coerce').fillna(0)
    df_exclusiva['TEA'] = pd.to_numeric(df_exclusiva['TEA'], errors='coerce').fillna(0)
    
    # Group by all columns except TEA and sum the TEA values
    group_columns = ['Regiao', 'Estado', 'Municipio', 'cod_Municipio']
    df = pd.concat([df_comum, df_exclusiva]).groupby(group_columns, as_index=False)['TEA'].sum()
    
    # Filter and set index
    df_filtrado = df.set_index("Regiao")
    
    # Process municipalities
    mask = df_filtrado['cod_Municipio'].apply(lambda x: isinstance(x, numbers.Number))
    df_municipios = df_filtrado[mask][["Municipio", "cod_Municipio", "TEA"]]
    df_municipios['Município'] = df_municipios.apply(
        lambda x: f"{int(x['cod_Municipio'])} {x['Municipio']}", axis=1
    )
    
    # Sort by municipality code before setting index
    df_municipios['sort_key'] = df_municipios['cod_Municipio'].astype(float)
    df_municipios = df_municipios.sort_values('sort_key')
    df_municipios = df_municipios.drop('sort_key', axis=1)
    
    # Final processing and save
    df_municipios = (df_municipios[['Município', 'TEA']]
                    .drop_duplicates(subset=['Município'])
                    .set_index('Município'))
    #df_municipios['TEA'] = df_municipios['TEA'].replace({'': '-', '0': '-', 0: '-'})
    df_municipios['TEA'] = df_municipios['TEA'].replace({'': '0', '0': 0})

    if df_existing.empty:
        df_existing = pd.DataFrame(index=df_municipios.index)
    
    df_existing[year] = df_municipios['TEA']
    df_existing.to_csv("data-out/data-TEA-matriculas.csv")

print(file)


Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2014.xlsx for year 2014
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2015.xlsx for year 2015
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2016.xlsx for year 2016
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2017.xlsx for year 2017
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2018.xlsx for year 2018
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2019.xlsx for year 2019
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2020.xlsx for year 2020
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2021.xlsx for year 2021
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2022.xlsx for year 2022
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx for year 2023
data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx


In [10]:
try:
    df_existing = pd.read_csv("data-out/data-Total_de_matriculas-matriculas.csv", index_col='Ano')
except FileNotFoundError:
    df_existing = pd.DataFrame()

for file in sorted(glob.glob("data-src/*.xlsx")):
    year = extract_year(file)
    
    if year in df_existing.columns:
        print(f"Data for year {year} already exists in the dataset.")
        continue

    print(f"Processing file: {file} for year {year}")
    wb = CalamineWorkbook.from_path(file)
    
    # Define sheet names and columns once
    aba_comum = "Educação Básica 1.1"
    columns_to_keep = ["Regiao", "Estado", "Municipio", "cod_Municipio", "Total_de_matriculas"]

    def process_df_rename(data):
        df = pd.DataFrame(data[9:], columns=data[7])
        df.columns.values[0]= "Regiao"
        df.columns.values[1]= "Estado" 
        df.columns.values[2] = "Municipio"
        df.columns.values[3] = "cod_Municipio"
        df.columns.values[4] = "Total_de_matriculas"
        return df[columns_to_keep]

    # Process both sheets
    df_comum = process_df_rename(wb.get_sheet_by_name(aba_comum).to_python())
    
    # Convert TEA to numeric, replacing non-numeric values with 0
    #df_comum['Total_de_matriculas'] = pd.to_numeric(df_comum['TEA'], errors='coerce').fillna(0)
    #df_exclusiva['TEA'] = pd.to_numeric(df_exclusiva['TEA'], errors='coerce').fillna(0)
    
    # Group by all columns except TEA and sum the TEA values
    #group_columns = ['Regiao', 'Estado', 'Municipio', 'cod_Municipio']
    #df = pd.concat([df_comum, df_exclusiva]).groupby(group_columns, as_index=False)['TEA'].sum()
    
    # Filter and set index
    df_filtrado = df_comum.set_index("Regiao")
    
    # Process municipalities
    mask = df_filtrado['cod_Municipio'].apply(lambda x: isinstance(x, numbers.Number))
    df_municipios = df_filtrado[mask][["Municipio", "cod_Municipio", "Total_de_matriculas"]]
    df_municipios['Município'] = df_municipios.apply(
        lambda x: f"{int(x['cod_Municipio'])} {x['Municipio']}", axis=1
    )
    
    # Sort by municipality code before setting index
    df_municipios['sort_key'] = df_municipios['cod_Municipio'].astype(float)
    df_municipios = df_municipios.sort_values('sort_key')
    df_municipios = df_municipios.drop('sort_key', axis=1)
    
    # Final processing and save
    df_municipios = (df_municipios[['Município', 'Total_de_matriculas']]
                    .drop_duplicates(subset=['Município'])
                    .set_index('Município'))
    #df_municipios['TEA'] = df_municipios['TEA'].replace({'': '-', '0': '-', 0: '-'})
    df_comum['Total_de_matriculas'] = pd.to_numeric(df_comum['Total_de_matriculas'], errors='coerce').fillna(0)
    #df_municipios['Total_de_matriculas'] = df_municipios['Total_de_matriculas'].replace({'': '0', '0': 0})

    if df_existing.empty:
        df_existing = pd.DataFrame(index=df_municipios.index)
    
    df_existing[year] = df_municipios['Total_de_matriculas']
    df_existing.to_csv("data-out/data-Total_de_matriculas-matriculas.csv")

print(file)


Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2014.xlsx for year 2014
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2015.xlsx for year 2015
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2016.xlsx for year 2016
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2017.xlsx for year 2017
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2018.xlsx for year 2018
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2019.xlsx for year 2019
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2020.xlsx for year 2020
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2021.xlsx for year 2021
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2022.xlsx for year 2022
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx for year 2023
data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx


# MATRICULAS TOTAIS ESTADOS

In [11]:
try:
    df_existing = pd.read_csv("data-out/data-Total_de_matriculas.csv", index_col='Ano')
except FileNotFoundError:
    df_existing = pd.DataFrame()

# Process all Excel files in data directory
for file in sorted(glob.glob("data-src/*.xlsx")):
    year = extract_year(file)
    
    # Skip if year already exists in df_existing
    if year in df_existing.index:
        print(f"Data for year {year} already exists in the dataset.")
        continue
    # Set sheet name based on year
    aba = "Educação Básica 1.1"
    
    print(f"Processing file: {file} for year {year}")
    wb = CalamineWorkbook.from_path(file)
    data = wb.get_sheet_by_name(aba).to_python()
    df = pd.DataFrame(data[9:], columns=data[7])
    
    df.columns.values[0]= "Regiao"
    df.columns.values[1]= "Estado" 
    df.columns.values[2] = "Municipio"
    df.columns.values[3] = "cod_Municipio"
    df.columns.values[4] = "Total_de_matriculas"

    df = df.loc[:, ["Regiao","Estado","Municipio","cod_Municipio","Total_de_matriculas"]]
    df = df.set_index("Regiao")
    dados_validos = (df["Total_de_matriculas"]!='')
    df_filtrado = df[dados_validos]

    # Filter states according to the predefined list
    df_estados = df_filtrado[df_filtrado['Estado'].str.strip().isin(estados)]
    df_estados = df_estados.loc[:, ["Estado","Total_de_matriculas"]]
    
    # Create pivot table with states as index
    df_pivot = df_estados.pivot_table(index='Estado', values='Total_de_matriculas', aggfunc='first')
    
    if df_existing.empty:
        df_existing = df_pivot
        df_existing.columns = [year]
    else:
        df_existing[year] = df_pivot['Total_de_matriculas']
    
    # Export to CSV
    df_existing.to_csv("data-out/data-Total_de_matriculas-estados.csv")

print(file)

Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2014.xlsx for year 2014
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2015.xlsx for year 2015
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2016.xlsx for year 2016
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2017.xlsx for year 2017
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2018.xlsx for year 2018
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2019.xlsx for year 2019
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2020.xlsx for year 2020
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2021.xlsx for year 2021
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2022.xlsx for year 2022
Processing file: data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx for year 2023
data-src/Sinopse_Estatistica_da_Educação_Basica_2023.xlsx


# MATRICULAS TOTAIS MUNICIPIOS

In [17]:
try:
    df_existing = pd.read_csv("data/data-Total_de_matriculas-municipio.csv", index_col='Ano')
except FileNotFoundError:
    df_existing = pd.DataFrame()

# Process all Excel files in data directory
for file in sorted(glob.glob("data-src/*.xlsx")):
    year = extract_year(file)
    
    # Skip if year already exists in df_existing
    if year in df_existing.columns:
        print(f"Data for year {year} already exists in the dataset.")
        continue
    # Set sheet name based on year
    aba = "1.11"
    print(f"Processing file: {file} for year {year}")
    wb = CalamineWorkbook.from_path(file)
    data = wb.get_sheet_by_name(aba).to_python()
    df = pd.DataFrame(data[9:], columns=data[7])
    
    df.columns.values[0]= "Regiao"
    df.columns.values[1]= "Estado"
    df.columns.values[2] = "Municipio"
    df.columns.values[3] = "cod_Municipio"
    df.columns.values[4] = "Total_de_matriculas"
    
    df = df.loc[:, ["Regiao","Estado","Municipio","cod_Municipio","Total_de_matriculas"]]
    df = df.set_index("Regiao")
    dados_validos = (df["Total_de_matriculas"]!='')
    df_filtrado = df[dados_validos]

    so_estados = df_filtrado["cod_Municipio"].apply(lambda x: isinstance(x, numbers.Number))
    df_filtrado_estado = df_filtrado[~so_estados]
    df_filtrado_estado = df_filtrado_estado.loc[:, ["Estado","Total_de_matriculas"]]

    # Filter municipalities for all states
    df_municipios = df_filtrado[df_filtrado['cod_Municipio'].apply(lambda x: isinstance(x, numbers.Number))]
    df_municipios = df_municipios.loc[:, ["Municipio","cod_Municipio","Total_de_matriculas"]]
    # Create new column combining code and name
    df_municipios['Município'] = df_municipios.apply(lambda x: f"{int(x['cod_Municipio'])} {x['Municipio']}", axis=1)
    # Drop original columns and keep only TEA and new combined column
    df_municipios = df_municipios[['Município', 'Total_de_matriculas']]
    # Sort by municipality code (extracted from the combined column)
    df_municipios['sort_key'] = df_municipios['Município'].str.extract(r'(\d+)').astype(int)
    df_municipios = df_municipios.sort_values('sort_key')
    df_municipios = df_municipios.drop('sort_key', axis=1)
    # Remove duplicates before setting index
    df_municipios = df_municipios.drop_duplicates(subset=['Município'])
    df_municipios = df_municipios.set_index("Município")
    # Replace empty or zero values with '-'
    #df_municipios['Total_de_matriculas'] = df_municipios['Total_de_matriculas'].replace({'': '-', '0': '-', 0: '-'})
    # Convert empty strings and '0' to numeric zeros, then ensure float type
    df_municipios['Total_de_matriculas'] = pd.to_numeric(df_municipios['Total_de_matriculas'].replace({'': '0', '0': '0'}), errors='coerce').fillna(0)
    
    if df_existing.empty:
        df_existing = pd.DataFrame(index=df_municipios.index)
    
    df_existing[year] = df_municipios['Total_de_matriculas']
    
    # Export to CSV
    df_existing.to_csv("data-out/data-Total_de_matriculas-municipio.csv")

print(file)

Processing file: data-src\Sinopse_Estatistica_da_Educação_Basica_2014.xlsx for year 2014


C:\Users\Luisgab\AppData\Local\Temp\ipykernel_8360\660679555.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_municipios['Total_de_matriculas'] = pd.to_numeric(df_municipios['Total_de_matriculas'].replace({'': '0', '0': '0'}), errors='coerce').fillna(0)


Processing file: data-src\Sinopse_Estatistica_da_Educação_Basica_2015.xlsx for year 2015


C:\Users\Luisgab\AppData\Local\Temp\ipykernel_8360\660679555.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_municipios['Total_de_matriculas'] = pd.to_numeric(df_municipios['Total_de_matriculas'].replace({'': '0', '0': '0'}), errors='coerce').fillna(0)


Processing file: data-src\Sinopse_Estatistica_da_Educação_Basica_2016.xlsx for year 2016


...

C:\Users\Luisgab\AppData\Local\Temp\ipykernel_8360\660679555.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_municipios['Total_de_matriculas'] = pd.to_numeric(df_municipios['Total_de_matriculas'].replace({'': '0', '0': '0'}), errors='coerce').fillna(0)


Processing file: data-src\Sinopse_Estatistica_da_Educação_Basica_2022.xlsx for year 2022


C:\Users\Luisgab\AppData\Local\Temp\ipykernel_8360\660679555.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_municipios['Total_de_matriculas'] = pd.to_numeric(df_municipios['Total_de_matriculas'].replace({'': '0', '0': '0'}), errors='coerce').fillna(0)


Processing file: data-src\Sinopse_Estatistica_da_Educação_Basica_2023.xlsx for year 2023
data-src\Sinopse_Estatistica_da_Educação_Basica_2023.xlsx


C:\Users\Luisgab\AppData\Local\Temp\ipykernel_8360\660679555.py:53: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_municipios['Total_de_matriculas'] = pd.to_numeric(df_municipios['Total_de_matriculas'].replace({'': '0', '0': '0'}), errors='coerce').fillna(0)
